# Imports

In [1]:
import numpy as np
import tensorflow as tf

/home/parallels/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# use preprocess function from evaluation preparation

In [2]:
def preprocess_for_eval(image, height, width,
                        central_fraction=0.875, scope=None):
    """Prepare one image for evaluation.

    If height and width are specified it would output an image with that size by
    applying resize_bilinear.

    If central_fraction is specified it would crop the central fraction of the
    input image.

    Args:
        image: 3-D Tensor of image. If dtype is tf.float32 then the range should be
        [0, 1], otherwise it would converted to tf.float32 assuming that the range
        is [0, MAX], where MAX is largest positive representable number for
        int(8/16/32) data type (see `tf.image.convert_image_dtype` for details).
        height: integer
        width: integer
        central_fraction: Optional Float, fraction of the image to crop.
        scope: Optional scope for name_scope.
    Returns:
        3-D float Tensor of prepared image.
    """
    with tf.name_scope(scope, 'eval_image', [image, height, width]):
        if image.dtype != tf.float32:
            image = tf.image.convert_image_dtype(image, dtype=tf.float32)
        # Crop the central region of the image with an area containing 87.5% of
        # the original image.
        if central_fraction:
            image = tf.image.central_crop(image, central_fraction=central_fraction)

        if height and width:
        # Resize the image to the specified height and width.
            image = tf.expand_dims(image, 0)
            image = tf.image.resize_bilinear(image, [height, width],
                                             align_corners=False)
            image = tf.squeeze(image, [0])
        image = tf.subtract(image, 0.5)
        image = tf.multiply(image, 2.0)
        return image

# Load the model and make inference

In [3]:
# Load a (frozen) Tensorflow model into memory.
with open('./pj_vehicle_inception_v4_freeze.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

# read a image into a numpy array
image_path = './test.jpg'
image_file = tf.read_file(image_path)
image = tf.image.decode_jpeg(image_file)
# prepare the image into a fixed size
image_preprocessed = preprocess_for_eval(image, 299, 299)
image_np_expanded = np.expand_dims(tf.Session().run(image_preprocessed), axis=0)
# make inference
sess = tf.Session()
softmax_tensor = sess.graph.get_tensor_by_name('InceptionV4/Logits/Predictions:0')
predictions = sess.run(softmax_tensor, feed_dict={'input:0': image_np_expanded})
predictions = np.squeeze(predictions)

# output the class index and score

### result for nocar.jpg

In [4]:
np.where(predictions == np.max(predictions))[0][0]

356

In [5]:
np.max(predictions)

0.18913023

### result for test.jpg

In [4]:
np.where(predictions == np.max(predictions))[0][0]

349

In [5]:
np.max(predictions)

0.90081704